<a href="https://colab.research.google.com/github/carbotton/price_scraper/blob/main/price_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 🔁 Reinstalamos Chrome para que coincida con versión 137
!wget -q -O chrome.deb https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i chrome.deb
!apt -f install -y > /dev/null

# 🧠 Detectamos versión actual
!google-chrome --version

Selecting previously unselected package google-chrome-stable.
(Reading database ... 126111 files and directories currently installed.)
Preparing to unpack chrome.deb ...
Unpacking google-chrome-stable (137.0.7151.68-1) ...
dpkg: dependency problems prevent configuration of google-chrome-stable:
 google-chrome-stable depends on libvulkan1; however:
  Package libvulkan1 is not installed.

dpkg: error processing package google-chrome-stable (--install):
 dependency problems - leaving unconfigured
Processing triggers for mailcap (3.70+nmu1ubuntu1) ...
Processing triggers for man-db (2.10.2-1) ...
Errors were encountered while processing:
 google-chrome-stable


Google Chrome 137.0.7151.68 


In [ ]:
!wget -q https://storage.googleapis.com/chrome-for-testing-public/137.0.7151.55/linux64/chromedriver-linux64.zip
!unzip -q chromedriver-linux64.zip
!mv chromedriver-linux64/chromedriver /usr/bin/chromedriver
!chmod +x /usr/bin/chromedriver

In [ ]:
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.14.0
    Uninstalling typing_extensions-4.14.0:
      Successfully uninstalled typing_extensions-4.14.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 r

# Inicializar drive de selenium

In [ ]:
def init_driver():
  from selenium import webdriver
  from selenium.webdriver.chrome.service import Service
  from selenium.webdriver.chrome.options import Options
  from selenium.webdriver.common.by import By
  import time

  chrome_options = Options()
  chrome_options.add_argument("--headless")
  chrome_options.add_argument("--no-sandbox")
  chrome_options.add_argument("--disable-dev-shm-usage")
  chrome_options.add_argument("--window-size=1920,1080")

  driver = webdriver.Chrome(service=Service("/usr/bin/chromedriver"), options=chrome_options)

  return driver


In [ ]:
from selenium.webdriver.common.by import By

# Ir directamente a la página de arroz
driver = init_driver()
driver.get("https://www.disco.com.uy/arroz")
time.sleep(5)  # o usar WebDriverWait como ya vimos

# Buscar imágenes con clase que contiene los productos
imagenes = driver.find_elements(By.CSS_SELECTOR, "img.vtex-product-summary-2-x-image")

# Extraer nombres desde alt
for img in imagenes:
    nombre = img.get_attribute("alt")
    if nombre:
        print("🛒", nombre)


NameError: name 'time' is not defined

In [ ]:
from selenium.webdriver.common.by import By

driver.get("https://www.disco.com.uy/arroz")
time.sleep(5)

# Obtener imágenes con el nombre del producto en el alt
imagenes = driver.find_elements(By.CSS_SELECTOR, "img.vtex-product-summary-2-x-image")

# Obtener todos los bloques de precio
bloques_precio = driver.find_elements(By.CSS_SELECTOR, "span.devotouy-products-components-0-x-sellingPriceWithUnitMultiplier")

# Emparejar nombre y precio (asumiendo que están en el mismo orden)
for i in range(min(len(imagenes), len(bloques_precio))):
    nombre = imagenes[i].get_attribute("alt")
    try:
        monto = bloques_precio[i].find_elements(By.TAG_NAME, "span")[-1].text
    except:
        monto = "N/A"
    print(f"🛒 {nombre} - 💰 ${monto}")


NameError: name 'time' is not defined

El codigo anterior solamente carga una parte de los elementos. Hay que hacer scroll y esperar a que carguen los demas elementos (lazy loading)

In [ ]:
import time
from selenium.webdriver.common.by import By

# Ir a la página de arroz
driver.get("https://www.disco.com.uy/arroz")
time.sleep(3)

# ⬇️ Scroll dinámico hasta que no carguen más productos
last_height = driver.execute_script("return document.body.scrollHeight")
scroll_tries = 0

while scroll_tries < 10:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    new_height = driver.execute_script("return document.body.scrollHeight")

    if new_height == last_height:
        scroll_tries += 1
    else:
        scroll_tries = 0
        last_height = new_height

# 🛒 Extraer nombres
imagenes = driver.find_elements(By.CSS_SELECTOR, "img.vtex-product-summary-2-x-image")
bloques_precio = driver.find_elements(By.CSS_SELECTOR, "span.devotouy-products-components-0-x-sellingPriceWithUnitMultiplier")

# Mostrar resultados
for i in range(min(len(imagenes), len(bloques_precio))):
    nombre = imagenes[i].get_attribute("alt")
    try:
        monto = bloques_precio[i].find_elements(By.TAG_NAME, "span")[-1].text
    except:
        monto = "N/A"
    print(f"🛒 {nombre} - 💰 ${monto}")

driver.quit()


# El que funciono para TATA

In [ ]:
import time
from selenium.webdriver.common.by import By
from selenium import webdriver

# El driver se inicializo mas arriba

driver.maximize_window()

# --- 2) Ir a la página de arroces de Tata ---
driver.get("https://www.tata.com.uy/arroz")  # https://www.tata.com.uy/s/?q=frutos+secos
time.sleep(3)

# --- 3) Hacer scroll dinámico para cargar todos los productos ---
last_height = driver.execute_script("return document.body.scrollHeight")
scroll_tries = 0

while scroll_tries < 10:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    new_height = driver.execute_script("return document.body.scrollHeight")

    if new_height == last_height:
        scroll_tries += 1
    else:
        scroll_tries = 0
        last_height = new_height

# --- 4) Localizar todos los bloques de producto ---
productos = driver.find_elements(By.CSS_SELECTOR, "section[data-product-card-content]")

# --- 5) Iterar y extraer nombre, marca y precio (oferta > original) ---
for producto in productos:
    # 5.1) Nombre del producto
    try:
        nombre_elem = producto.find_element(By.CSS_SELECTOR, "h3[data-fs-product-card-title] a")
        nombre = nombre_elem.text.strip()
    except:
        nombre = "N/A"

    # 5.2) Marca
    try:
        marca_elem = producto.find_element(By.CSS_SELECTOR, "div[data-fs-product-card-brand] p.product-shelf__brand-text")
        marca = marca_elem.text.strip()
    except:
        marca = "N/A"

    # 5.3) Precio: primero intento leer precio de oferta (spot). Si no existe, leo precio original (listing)
    precio_valor = "N/A"
    precio_display = "N/A"

    # Busco todos los spans que coincidan con precio de oferta
    ofertas = producto.find_elements(By.CSS_SELECTOR, "span[data-testid='price'][data-variant='spot']")
    if ofertas:
        # Tomo el primero (suele haber uno solo)
        precio_elem = ofertas[0]
        precio_valor = precio_elem.get_attribute("data-value")   # valor numérico, ej. "59.25"
        precio_display = precio_elem.text.strip()                # texto formateado, ej. "$ 59,25"
    else:
        # Si no hay precio de oferta, busco precio original (list price)
        originales = producto.find_elements(By.CSS_SELECTOR, "span[data-testid='list-price'][data-variant='listing']")
        if originales:
            precio_elem = originales[0]
            precio_valor = precio_elem.get_attribute("data-value")   # ej. "79"
            precio_display = precio_elem.text.strip()                # ej. "$ 79,00"
        # Si tampoco hay precio original, queda en "N/A"

    print(f"Nombre: {nombre} | Marca: {marca} | Precio: {precio_display} (valor numérico: {precio_valor})")

# --- 6) Cerrar el driver al finalizar ---
driver.quit()


MaxRetryError: HTTPConnectionPool(host='localhost', port=35491): Max retries exceeded with url: /session/a159da43f4acc304ab5a455a4fd37547/window/maximize (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7c74d0b12850>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [ ]:
!which pip
!which python3
import sys
print(sys.executable)

# Data frames

## df_inventario
Precios de todos los items de todos los supermercados.

## df_surtido
Lista de compras

## df_donde_comprar
En que supermercado comprar cada item

# Ejemplo para Disco y arroz:

In [ ]:
import pandas as pd
import time
from selenium.webdriver.common.by import By

# Inicializar lista donde guardaremos los datos
scraped_data = []

# Ir a la página de arroz
driver.get("https://www.disco.com.uy/arroz")
time.sleep(3)

# ⬇️ Scroll dinámico hasta que no carguen más productos
last_height = driver.execute_script("return document.body.scrollHeight")
scroll_tries = 0

while scroll_tries < 10:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    new_height = driver.execute_script("return document.body.scrollHeight")

    if new_height == last_height:
        scroll_tries += 1
    else:
        scroll_tries = 0
        last_height = new_height

# 🛒 Extraer nombres e íconos de producto
imagenes = driver.find_elements(By.CSS_SELECTOR, "img.vtex-product-summary-2-x-image")
# 🏷️ Extraer bloques de precio
bloques_precio = driver.find_elements(By.CSS_SELECTOR, "span.devotouy-products-components-0-x-sellingPriceWithUnitMultiplier")

# Recorrer elementos y guardar en scraped_data
for i in range(min(len(imagenes), len(bloques_precio))):
    nombre = imagenes[i].get_attribute("alt")
    try:
        # El precio real está en el último <span> dentro de cada bloque de precio
        monto_texto = bloques_precio[i].find_elements(By.TAG_NAME, "span")[-1].text
        # Convertir monto a número (suponiendo que viene sin separador de miles y sin moneda)
        monto = float(monto_texto.replace(',', '').replace('$', '').strip())
    except:
        monto = None

    scraped_data.append({
        'Item': nombre,
        'Supermercado': 'Disco',
        'Precio': monto
    })

# Crear DataFrame con los datos scrapeados
df_inventario = pd.DataFrame(scraped_data, columns=['Item', 'Supermercado', 'Precio'])

# Mostrar el DataFrame para verificar el resultado
display(df_inventario)


,Item,Supermercado,Precio
0,Arroz patna BLUE PATNA 1 kg,Disco,79.0
1,Arroz patna BLUE PATNA 5 kg,Disco,249.0
2,Arroz patna green CHEF 5 kg,Disco,239.0
3,Arroz blanco MIROKUMAI 1 kg,Disco,152.0
4,Arroz integral orgánico cocido CAMPOCLARO 250 g,Disco,164.0
...,...,...,...
60,Bebida THE BRIDGE de Arroz Tostado Bio 1 L,Disco,259.0
61,Bebida THE BRIDGE bio de arroz y coco 250 ml,Disco,111.0
62,Cebolla en aros Club Verde 350 g,Disco,76.0
63,Galletas de arroz RICE UP Mushrooms 50 g,Disco,83.0


In [ ]:
from urllib.parse import quote, quote_plus

def encode_producto(base_url: str, producto: str) -> str:
    """
    Dada la URL base y el nombre de producto (ej: "frutos secos"),
    devuelve el string codificado apropiadamente:
    - Disco  -> espacios -> %20
    - Tata   -> espacios -> +
    """
    if "disco" in base_url.lower():
        return quote(producto)          # p.ej. "frutos%20secos"
    elif "tata" in base_url.lower():
        return quote_plus(producto)     # p.ej. "frutos+secos"
    else:
        # Si hay otras tiendas u otros formatos, podés agregar más casos aquí
        return quote(producto)  # fallback: usar siempre %20


In [ ]:
import pandas as pd
import time
from selenium.webdriver.common.by import By

# Inicializar lista donde guardaremos los datos
scraped_data = []

# Ir a la página de arroz
base_url_list = [
    "https://www.tata.com.uy/s/?q=",
    "https://www.disco.com.uy/"
    ]

product_list = [
    "arroz",
    "frutos secos",
    "atun",
    "mermelada"]

for base_url in base_url_list:
  for product in product_list:
    prod_encoded = encode_producto(base_url, product)
    driver.get(base_url+prod_encoded)
    time.sleep(3)

    # ⬇️ Scroll dinámico hasta que no carguen más productos
    last_height = driver.execute_script("return document.body.scrollHeight")
    scroll_tries = 0

    while scroll_tries < 10:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:
            scroll_tries += 1
        else:
            scroll_tries = 0
            last_height = new_height

    # 🛒 Extraer nombres e íconos de producto
    imagenes = driver.find_elements(By.CSS_SELECTOR, "img.vtex-product-summary-2-x-image")
    # 🏷️ Extraer bloques de precio
    bloques_precio = driver.find_elements(By.CSS_SELECTOR, "span.devotouy-products-components-0-x-sellingPriceWithUnitMultiplier")

    # Recorrer elementos y guardar en scraped_data
    for i in range(min(len(imagenes), len(bloques_precio))):
        nombre = imagenes[i].get_attribute("alt")
        try:
            # El precio real está en el último <span> dentro de cada bloque de precio
            monto_texto = bloques_precio[i].find_elements(By.TAG_NAME, "span")[-1].text
            # Convertir monto a número (suponiendo que viene sin separador de miles y sin moneda)
            monto = float(monto_texto.replace(',', '').replace('$', '').strip())
        except:
            monto = None

        scraped_data.append({
            'Item': nombre,
            'Supermercado': 'Disco',
            'Precio': monto
        })

    # Crear DataFrame con los datos scrapeados
    df_inventario = pd.DataFrame(scraped_data, columns=['Item', 'Supermercado', 'Precio'])

# Mostrar el DataFrame para verificar el resultado
display(df_inventario)

KeyboardInterrupt: 

# TATA y Disco

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time

def encode_producto(base_url, product):
    """
    Codifica espacios según cada supermercado:
    - Tata: reemplaza espacios por '+'
    - Disco: reemplaza espacios por '%20'
    """
    if "tata.com.uy" in base_url:
        return product.replace(' ', '+')
    else:  # disco.com.uy
        return product.replace(' ', '%20')

def init_driver():
  from selenium import webdriver
  from selenium.webdriver.chrome.service import Service
  from selenium.webdriver.chrome.options import Options
  from selenium.webdriver.common.by import By
  import time

  chrome_options = Options()
  chrome_options.add_argument("--headless")
  chrome_options.add_argument("--no-sandbox")
  chrome_options.add_argument("--disable-dev-shm-usage")
  chrome_options.add_argument("--window-size=1920,1080")

  driver = webdriver.Chrome(service=Service("/usr/bin/chromedriver"), options=chrome_options)

  return driver

driver = init_driver()
driver.maximize_window()

scraped_data = []
base_url_list = [
    "https://www.tata.com.uy/s/?q=",
    "https://www.disco.com.uy/"
]
product_list = ["arroz", "frutos secos", "atun", "mermelada"]

for base_url in base_url_list:
    for product in product_list:
        prod_encoded = encode_producto(base_url, product)
        url = base_url + prod_encoded
        driver.get(url)

        # Identificamos de cuál supermercado se trata
        if "tata.com.uy" in base_url:
            supermercado = "Tata"
            # Esperamos a que aparezca al menos un bloque de producto de Tata
            try:
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located(
                        (By.CSS_SELECTOR, "section[data-product-card-content]")
                    )
                )
            except TimeoutException:
                print(f"No cargaron productos de Tata para '{product}' en {url}")
                continue

            # Scroll dinámico (3 intentos sin cambios, 1s de espera)
            last_height = driver.execute_script("return document.body.scrollHeight")
            scroll_intentos = 0
            while scroll_intentos < 3:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(1)
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    scroll_intentos += 1
                else:
                    scroll_intentos = 0
                    last_height = new_height

            # Extraer todos los bloques de producto en Tata
            productos = driver.find_elements(By.CSS_SELECTOR, "section[data-product-card-content]")
            for producto_elem in productos:
                # 1) Nombre
                try:
                    nombre_elem = producto_elem.find_element(
                        By.CSS_SELECTOR, "h3[data-fs-product-card-title] a"
                    )
                    nombre = nombre_elem.text.strip()
                except:
                    nombre = "N/A"

                # 2) Precio (oferta > original)
                precio_valor = None
                # Primero buscamos precio de oferta (spot)
                ofertas = producto_elem.find_elements(
                    By.CSS_SELECTOR, "span[data-testid='price'][data-variant='spot']"
                )
                if ofertas:
                    precio_raw = ofertas[0].get_attribute("data-value")  # ej. "59.25"
                    try:
                        precio_valor = float(precio_raw)
                    except:
                        precio_valor = None
                else:
                    # Si no hay oferta, buscamos precio original (listing)
                    originales = producto_elem.find_elements(
                        By.CSS_SELECTOR, "span[data-testid='list-price'][data-variant='listing']"
                    )
                    if originales:
                        precio_raw = originales[0].get_attribute("data-value")  # ej. "79"
                        try:
                            precio_valor = float(precio_raw)
                        except:
                            precio_valor = None

                scraped_data.append({
                    'Item': nombre,
                    'Supermercado': supermercado,
                    'Precio': precio_valor
                })

        else:
            # ------------------- Disco -------------------
            supermercado = "Disco"
            # Esperamos a que cargue al menos una imagen de producto de Disco
            try:
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located(
                        (By.CSS_SELECTOR, "img.vtex-product-summary-2-x-image")
                    )
                )
            except TimeoutException:
                print(f"No cargaron productos de Disco para '{product}' en {url}")
                continue

            # Scroll dinámico (3 intentos sin cambios, 1s de espera)
            last_height = driver.execute_script("return document.body.scrollHeight")
            scroll_intentos = 0
            while scroll_intentos < 3:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(1)
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    scroll_intentos += 1
                else:
                    scroll_intentos = 0
                    last_height = new_height

            # Extraer imágenes y bloques de precio de Disco
            imagenes = driver.find_elements(By.CSS_SELECTOR, "img.vtex-product-summary-2-x-image")
            bloques_precio = driver.find_elements(
                By.CSS_SELECTOR,
                "span.devotouy-products-components-0-x-sellingPriceWithUnitMultiplier"
            )

            # Iterar sobre los pares (imagen, bloque_precio)
            for i in range(min(len(imagenes), len(bloques_precio))):
                nombre = imagenes[i].get_attribute("alt").strip()
                try:
                    # El último <span> dentro del bloque_precio contiene el número
                    monto_texto = bloques_precio[i].find_elements(By.TAG_NAME, "span")[-1].text
                    # Convertimos "$ 79" o "79" a float
                    monto = float(monto_texto.replace(',', '').replace('$', '').strip())
                except:
                    monto = None

                scraped_data.append({
                    'Item': nombre,
                    'Supermercado': supermercado,
                    'Precio': monto
                })

# Una vez fuera de los bucles, creamos el DataFrame final
df_inventario = pd.DataFrame(scraped_data, columns=['Item', 'Supermercado', 'Precio'])
display(df_inventario)

# Cerrar el driver al finalizar
driver.quit()


,Item,Supermercado,Precio
0,Arroz Patna Green Chef Bolsa 1 Kg,Tata,60.00
1,Arroz Blanco Saman 1 Kg,Tata,67.00
2,Arroz Green Chef 5 Kg,Tata,249.00
3,Arroz Blanco Ta-Ta 1 Kg,Tata,68.00
4,Arroz Parboiled Ta-Ta 1 Kg,Tata,59.25
...,...,...,...
144,Mermelada LOS NIETITOS durazno intensa 390 g,Disco,208.00
145,Mermelada Frutilla Light CAMPOCLARO 360 g,Disco,281.00
146,Mermelada cereza SCHWARTAU 340 g,Disco,279.00
147,Mermelada RINCON DEL GIGANTE de frutilla dieté...,Disco,157.00


# Otros supermercados para agregar:
* Tienda Inglesa
* El Dorado
* El Clon

In [ ]:
import pandas as pd

# --- Asumimos que ya tienes df_inventario definido y cargado ---
# Normalizamos nombres en df_inventario
df_inventario['Item_norm'] = df_inventario['Item'].str.strip().str.lower()

# Tu lista de productos a comprar
product_list = ["arroz integral", "atun emigrante al natural 354", "Mermelada Durazno Los Nietitos 500 G"]

# Construimos el DataFrame de tu surtido
df_surtido = pd.DataFrame([{'Item': p} for p in product_list])

# Para cada producto, buscamos filas en df_inventario cuyo nombre contenga la palabra clave
results = []
for product in product_list:
    clave = product.strip().lower()
    # usamos str.contains para match parcial
    mask = df_inventario['Item_norm'].str.contains(clave, na=False)
    subset = df_inventario[mask]

    if subset.empty:
        # No se encontró ninguna coincidencia
        results.append({'Item': product, 'Supermercado': None, 'Precio': None})
    else:
        # Tomamos la fila con precio mínimo
        idx_min = subset['Precio'].idxmin()
        fila = subset.loc[idx_min]
        results.append({
            'Item':       product,
            'Supermercado': fila['Supermercado'],
            'Precio':       fila['Precio']
        })

df_donde_comprar = pd.DataFrame(results, columns=['Item', 'Supermercado', 'Precio'])

# Mostrar resultados
display(df_surtido)
display(df_donde_comprar)


,Item
0,arroz integral
1,atun emigrante al natural 354
2,Mermelada Durazno Los Nietitos 500 G


,Item,Supermercado,Precio
0,arroz integral,Tata,105.0
1,atun emigrante al natural 354,None,NaN
2,Mermelada Durazno Los Nietitos 500 G,Tata,125.0


 # TO-DO
 ## Hacer una funcion que contenga toda la parte de busqueda de precios y armado de inventario.
 Asi despues si en la lista de compras aparece algo que no estba en la lista de produtos original,
 que busque para ese precio en particular. No deberia agregarlo permanente a la product_list porque podria ser una compra unica.


 ## Ademas tendria que agregar una variable "descuentos extra", para casos como por ejemplo cuando hay descuento con mercado pago en tata.
 Ingreso el descuento, se lo aplica a todos los de tata y eso modifica el resultado de a donde ir a comprar.


 ## Si esto fuera una app:
 #### Variables configurables por el usuario
 * lista de compras
 * descuento extra (se selecciona por supermercado) (no se como entrarian aca los descuentos por usuario plus en tata o hipercoso en disco)
 * mapeo de productos: arroz = arroz integral 1 kg ; frutos secos = frutos secos sin sal ; azucar = azucar mascabo, cafe = recarga economica ; etc...

 #### Resultado
 En la pantalla aparece el nombre del supermercado en grande y abajo mas chico todos los supermercados con una lista al lado de los que deberias comprar en cada lugar para ahorrar mas.

